<a href="https://colab.research.google.com/github/kgautam2103/Bulk_Email_tool_public/blob/main/Timezone_dynamic_BulkEmailTool_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pytz
from datetime import datetime
from  pytz import timezone

output_format = ("%A, %B %d, %Y, %I:%M %p")
apac_EST = input("enter the apac time for EST like 2021-10-03 21:45 : ")
date_apac_EST = datetime.strptime(apac_EST, '%Y-%m-%d %H:%M')
naol_EST = input("enter the Naol time for EST like 2021-10-03 09:45 : ")
date_naol_EST = datetime.strptime(naol_EST, '%Y-%m-%d %H:%M')
#date_apac_EST = datetime.strptime(apac_EST, '%A, %B %d, %Y, %H:%M:%S')
est_timezone = pytz.timezone("America/New_York")
date_apac_EST_TZ = est_timezone.localize(date_apac_EST)
date_naol_EST_TZ = est_timezone.localize(date_naol_EST)
print(date_apac_EST_TZ)
print(date_naol_EST_TZ)


apac = ['Australia/Adelaide','Australia/Sydney','Australia/Perth','Asia/Shanghai','Asia/Tokyo','Asia/Bangkok','Asia/Kuala_Lumpur',
        'Asia/Manila','Asia/Singapore','Asia/Jakarta','Asia/Seoul']
naol= ['Europe/London','Europe/Rome']
apac_naol = ['America/New_York','America/Chicago','America/Denver','America/Los_Angeles','America/Halifax','Asia/Kolkata']

enter the apac time for EST like 2021-10-03 21:45 : 2021-12-26 21:45
enter the Naol time for EST like 2021-10-03 09:45 : 2021-12-26 09:45
2021-12-26 21:45:00-05:00
2021-12-26 09:45:00-05:00


In [ ]:
# Import Python Packages
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd
import time
import base64
from datetime import datetime
from email.message import EmailMessage
from email.utils import make_msgid
#from google.colab import files

# Set Global Variables
current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_email_output'
gmail_user = ''
gmail_password_encoded = ''

apac_link = "https://calendar.google.com/event?action=TEMPLATE&tmeid=MmhvaHM4NGpiOWdxb25kNjdnNDg2NnR2NjhfMjAyMTEwMTBUMDE0NTAwWiBwdnRoN2UzMzRobzk2MHJjdGVnaGYybWQ2c0Bn&tmsrc=pvth7e334ho960rcteghf2md6s%40group.calendar.google.com&scp=ALL"
naol_link = "https://calendar.google.com/event?action=TEMPLATE&tmeid=MHZmYXJuOHVicHRxcTNjbHZlcjZhOTJiZ3FfMjAyMTEwMTBUMTM0NTAwWiBwdnRoN2UzMzRobzk2MHJjdGVnaGYybWQ2c0Bn&tmsrc=pvth7e334ho960rcteghf2md6s%40group.calendar.google.com&scp=ALL"

is_embed_image=input('embed image needed ? Enter true or false: ').lower()
if is_embed_image == 'true':
    image_name = input('please enter image name like chakra.jpeg or chakra.jpg : ')

is_attachment=input('Attachment needed ? Enter true or false: ').lower()
if is_attachment == 'true':
  attached_file_name_with_path = input('enter the path along with attachment name like /content/MeditationGuide.pdf : ')
  last_index_slash = max(index for index, item in enumerate(attached_file_name_with_path) if item == '/')
  attached_file_name = attached_file_name_with_path[last_index_slash+1:]

counter = 1

base64_bytes = gmail_password_encoded.encode("ascii")  
pswd_string_bytes = base64.b64decode(base64_bytes)
gmail_password = pswd_string_bytes.decode("ascii")

#server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(gmail_user, gmail_password)
input_ws=input('please enter inputdata or sampledata: ')
#input_ws = 'sampledata'

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wSD-KgIqSbedwn2t8on9ovI6g_K6ZciktwfhB81T8rk/edit#gid=0')

input_sheet = wb.worksheet(input_ws)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['email','name','timezone','subject','message','regards','greeting'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['email_id','name', 'email_sent','error'],1)

#attach_file_name = '/content/MeditationGuide.pdf'
#attach_file = open(attach_file_name, 'rb') # Open the file as binary mode
#attach_file=files.upload()


for index, row in inputdata_df.iterrows():
  counter = counter+1
  from_email = gmail_user
  to_email = str(row['email'])
  email_subject = str(row['subject'])
  email_name = str(row['name'])
  email_tz = str(row['timezone']).strip()
  email_greet = str(row['greeting'])+" "+email_name+","
  email_message_body = str(row['message'])
  email_regards = str(row['regards'])
  email_success = ''
  email_error = ''
  event_link_string= "Please click and save below calendar event for Meditation event reminders"


  date_apac_TZ = date_apac_EST_TZ.astimezone(timezone(email_tz))
  date_str_apac_TZ = date_apac_TZ.strftime(output_format)
  date_naol_TZ = date_naol_EST_TZ.astimezone(timezone(email_tz))
  date_str_naol_TZ = date_naol_TZ.strftime(output_format)

  both_str_TZ = date_str_apac_TZ+" and "+date_str_naol_TZ

  if email_tz in apac:
    final_replace = date_str_apac_TZ
    final_link = date_str_apac_TZ+"\n"+apac_link
  elif email_tz in naol:
    final_replace = date_str_naol_TZ
    final_link = date_str_naol_TZ+"\n"+naol_link
  elif email_tz in apac_naol:
    final_replace = both_str_TZ
    final_link = date_str_apac_TZ+"\n"+apac_link+"\n\n"+date_str_naol_TZ+"\n"+naol_link

  email_subject = email_subject.replace('tm_replace',final_replace)
  email_message_body = email_message_body.replace('tm_replace',final_replace)
  email_message = '''
  %s
  <br><br>
  %s
  <br>
  %s
  <br>
  %s
  <br><br>
  Regards,
  <br>
  %s
  ''' % (email_greet, email_message_body, event_link_string, final_link, email_regards) 

  #print(email_message)

  #msg = MIMEMultipart()
  msg = EmailMessage()
  # setup the parameters of the message
  msg['From']=from_email
  msg['To']=to_email
  msg['Subject']=email_subject

  # add in the message body
  #msg.attach(MIMEText(email_message, 'plain'))
  msg.set_content(email_message,'html')


  if is_embed_image == 'true':
    #image_name = input('please enter image name like chakra.jpeg or chakra.jpg')
    unique_msg_id = make_msgid()
    message_body = """
        %s
        <br><br>
        %s
        <br>
        %s
        <br>
        %s
        <br>
        <img src="cid:%s" width="320" height="240" />
        <br><br><br>
        Regards,
        <br>
        %s
      </body>
    </html>
    """% (email_greet, email_message_body, event_link_string, final_link, unique_msg_id[1:-1],email_regards)
    msg.add_alternative(message_body, subtype='html')

    with open(image_name, 'rb') as img:
        msg.get_payload()[1].add_related(img.read(), 'image', 'jpeg',
                                        cid=unique_msg_id)
        
  #add attachment
  if is_attachment == 'true':
    #attach_file_name = '/content/MeditationGuide.pdf'
    attach_file = open(attached_file_name_with_path, 'rb') # Open the file as binary mode
    payload = MIMEBase('application', 'octate-stream')
    payload.set_payload((attach_file).read())
    encoders.encode_base64(payload) #encode the attachment
    #add payload header with filename
    payload.add_header('content-disposition', 'attachment', filename=attached_file_name)
    msg.attach(payload)

  try:
    # Sent Email  
    server.send_message(msg)
    #time.sleep(3)
  except Exception as e:
    email_success = 'False'
    email_error = str(e)
    print(e)
  else:
    email_success = 'True'
  finally:
    cell_value_list = [to_email,email_name,email_success,email_error]
    output_sheet.insert_row(cell_value_list,counter)

  del msg

server.close()

embed image needed ? Enter true or false: true
please enter image name like chakra.jpeg or chakra.jpg : 1.jpeg
Attachment needed ? Enter true or false: true
enter the path along with attachment name like /content/MeditationGuide.pdf : /content/dmv.pdf
please enter inputdata or sampledata: sampledata


In [ ]:
import base64
  
sample_string = ""
sample_string_bytes = sample_string.encode("ascii")
  
base64_bytes = base64.b64encode(sample_string_bytes)
base64_string = base64_bytes.decode("ascii")

base64_string

In [ ]:
print(row)

email              
name               
timezone           
subject            
message            
regards     SY Team
greeting       Dear
Name: 2, dtype: object
